#Moisés David Villalba Contreras
* Ingeniería de sistemas
* Deep Learning

#PLN Canciones de vallenato

##Instalando librerias CUDA

In [1]:
#!pip list
!nvcc --version  #Version de CUDA en la maquina virtual

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0


##P0. Importar librerías

In [2]:
import tensorflow as tf
import timeit               #para medir tiempos
import numpy as np
import pandas as pd 
import os
import time
import sys

###P0.1 Uso de GPU para entrenar en tensorflow


In [3]:
print("Tensorflow Version: ", tf.__version__)
print("Dispositivos disponibles para entrenar: ", tf.config.list_physical_devices())
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Encontrada la GPU: {}'.format(device_name))

Tensorflow Version:  2.7.0
Dispositivos disponibles para entrenar:  [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Encontrada la GPU: /device:GPU:0


###P0.2 probando rendimiento de CPU vs GPU

In [4]:
def cpu():
  with tf.device('/cpu:0'):
    random_image_cpu = tf.random.normal((100, 100, 100, 3))
    net_cpu = tf.keras.layers.Conv2D(32, 7)(random_image_cpu)
    return tf.math.reduce_sum(net_cpu)

def gpu():
  with tf.device('/device:GPU:0'):
    random_image_gpu = tf.random.normal((100, 100, 100, 3))
    net_gpu = tf.keras.layers.Conv2D(32, 7)(random_image_gpu)
    return tf.math.reduce_sum(net_gpu)   

In [5]:
cpu()  #ejecutamos entrenamiento con CPU
gpu()  #ejecutamos entrenamiento con GPU
# Run the op several times.
print('TIEMPO (seg) para entrenar una red convolucional de 32x7x7x3 filtros sobre un randomico de 100x100x100x3 imagenes '
      '(batch x height x width x channel). suma de 10 epochs.')
print('CPU (s):')
cpu_time = timeit.timeit('cpu()', number=10, setup="from __main__ import cpu")
print(cpu_time)
print('GPU (s):')
gpu_time = timeit.timeit('gpu()', number=10, setup="from __main__ import gpu")
print(gpu_time)
print('GPU speedup over CPU: {}x'.format(int(cpu_time/gpu_time)))

TIEMPO (seg) para entrenar una red convolucional de 32x7x7x3 filtros sobre un randomico de 100x100x100x3 imagenes (batch x height x width x channel). suma de 10 epochs.
CPU (s):
3.6904790179999907
GPU (s):
0.05349419699999203
GPU speedup over CPU: 68x


###P0.3 Dejando activo la GPU

In [6]:
tf.device('/device:GPU:0') #activando la GPU 

##P0. Descarga y preprocesado de los datos

In [7]:
fileDL= tf.keras.utils.get_file('Solo_Letras_PLN_Vallenato.txt','https://raw.githubusercontent.com/Moisesdv10/Deep-Learning/main/Proyecto_Final/Solo_Letras_PLN_Vallenato.txt')
texto = open(fileDL, 'rb').read().decode(encoding='utf-8')

196608/182264 [================================] - 0s 0us/step


###P0.1 limpiar/normalizar el texto

---
*   pasar todo a minusculas
*   convertir tildes a vocales sin tilde
*   eliminar caracteres especiales (*#!()%=
*   eliminar saltos de linea y tab (identación)

In [8]:
import re
from unicodedata import normalize
#pasa todo a minuscula
texto     = texto.lower()
#reemplazar tildes por letras similares sin tildes
transfor  = dict.fromkeys(map(ord, u'\u0301\u0308'), None)
texto     = normalize('NFKC', normalize('NFKD', texto).translate(transfor))
#quitar saltos de linea
texto      = texto.strip()
texto      = re.sub('\r|\n', ' ',texto)
#quitar espacios dobles
texto      = re.sub(' +', ' ', texto)
#quitando caracteres especiales
texto = re.sub(r"[^a-zA-Z0-9ñ]+"," ",texto)
print(texto)

no podras ahora cuentame tu que haces por aqui no digas que ese otro no te hizo feliz porque en mi tiempo solo de ti me olvide y ahora estoy junto a ella y me enamore mis amigos me dicen preguntas por mi ayer tu me tenias y hoy quieres volver no pierdas mas tu tiempo este amor se murio no seas tan ilusa que hoy me marcho yo no vengas a decirme que me amas mujer no te creo una palabra lo sabes muy bien flores versos y cartas mucho te obsequie y aun asi para ti suficiente no fue y ahora muy claramente te voy a decir cuando quieras besarme pues ya no podras estare en otra parte siendo muy feliz te estrellaras de frente pues no te amo mas alguien me brindara lo que tanto pedi y a ti te ignorare sin mirar hacia atras como me vas a amar si ni te amas a ti no me vas a engañar tiempo te dedique y descarta que a ti algun dia volvere hoy te digo que el tonto por fin se canso abro los ojos y miro con claridad tus desplantes un dia se iban a acabar y hoy te digo que el fin acaba de empezar me ire 

##P1. entendiendo el texto

In [9]:
print('el texto tiene longitud de:{} caracteres'. format(len(texto)))
vocab = sorted(set(texto))
print('el texto esta compuesto de estos :{} caracteres'. format(len(vocab)))
print(vocab)

el texto tiene longitud de:171059 caracteres
el texto esta compuesto de estos :35 caracteres
[' ', '0', '1', '2', '4', '6', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'ñ']


##P2. pasar el texto a números

---
as redes neuronales solo procesan valores numéricos, no letras, por tanto tenemos que traducir los caracteres a representación numérica. Para ello crearemos dos “tablas de traducción”: una de caracteres a números y otra de números a caracteres

In [10]:
char2idx = {u:i for i, u in enumerate(vocab)} # asignamos un número a cada vocablo
idx2char = np.array(vocab)
#-----------revisando las conversiones

#for char,_ in zip(char2idx, range(len(vocab))):
    #print(' {:4s}: {:3d},'.format(repr(char),char2idx[char]))

#pasamos todo el texto a números
texto_como_entero= np.array([char2idx[c] for c in texto])
print('texto: {}'.format(repr(texto[:100])))
print('{}'.format(repr(texto_como_entero[:100])))

texto: 'no podras ahora cuentame tu que haces por aqui no digas que ese otro no te hizo feliz porque en mi t'
array([21, 22,  0, 23, 22, 11, 25,  8, 26,  0,  8, 15, 22, 25,  8,  0, 10,
       28, 12, 21, 27,  8, 20, 12,  0, 27, 28,  0, 24, 28, 12,  0, 15,  8,
       10, 12, 26,  0, 23, 22, 25,  0,  8, 24, 28, 16,  0, 21, 22,  0, 11,
       16, 14,  8, 26,  0, 24, 28, 12,  0, 12, 26, 12,  0, 22, 27, 25, 22,
        0, 21, 22,  0, 27, 12,  0, 15, 16, 33, 22,  0, 13, 12, 19, 16, 33,
        0, 23, 22, 25, 24, 28, 12,  0, 12, 21,  0, 20, 16,  0, 27])


###P2.1 exportar vocablos y matriz de numerica

---

exporta en un CSV para usarlo cuando se cargue el modelo en otra instancia

In [11]:
rows=[]
columns=['num','vocab']
for i, voc in enumerate(vocab):
  #print(i,'-->', voc)
  rows.append([i,voc])
df= pd.DataFrame(columns=['num','vocab'],data=rows)
df.head(10)
df.to_csv('data_vocab.csv',index=False)

##P3. preparar los datos para ser usados en la RNN

In [12]:
char_dataset= tf.data.Dataset.from_tensor_slices(texto_como_entero)
#cantidad de secuencia de caracteres
secu_length=100
#creamos secuencias de maximo 100 caractereres
secuencias= char_dataset.batch(secu_length+1, drop_remainder=True)
for item in secuencias.take(10):
  print(repr(''.join(idx2char[item.numpy()])))

'no podras ahora cuentame tu que haces por aqui no digas que ese otro no te hizo feliz porque en mi ti'
'empo solo de ti me olvide y ahora estoy junto a ella y me enamore mis amigos me dicen preguntas por m'
'i ayer tu me tenias y hoy quieres volver no pierdas mas tu tiempo este amor se murio no seas tan ilus'
'a que hoy me marcho yo no vengas a decirme que me amas mujer no te creo una palabra lo sabes muy bien'
' flores versos y cartas mucho te obsequie y aun asi para ti suficiente no fue y ahora muy claramente '
'te voy a decir cuando quieras besarme pues ya no podras estare en otra parte siendo muy feliz te estr'
'ellaras de frente pues no te amo mas alguien me brindara lo que tanto pedi y a ti te ignorare sin mir'
'ar hacia atras como me vas a amar si ni te amas a ti no me vas a engañar tiempo te dedique y descarta'
' que a ti algun dia volvere hoy te digo que el tonto por fin se canso abro los ojos y miro con clarid'
'ad tus desplantes un dia se iban a acabar y hoy te digo que el 

###P3.1 separar los datos en agrupamientos (batches)

In [13]:
#funcion para obtener el conjunto de datos de trainning
def split_input_target(chunk):
  input_text = chunk[:-1]
  target_text= chunk[1:]
  return input_text, target_text

dataset  = secuencias.map(split_input_target)
#el dataset contiene un conjunto de parejas de secuencia de texto
#(con la representación numérica de los caracteres), donde el 
#primer componente de la pareja contiene un paquete con una secuencia 
#de 100 caracteres del texto original y la segunda su correspondiente salida, 
#también de 100 caracteres. )
for input_example, target_example in dataset.take(1):
  print('input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print('Target data: ', repr(''.join(idx2char[target_example.numpy()])))

input data:  'no podras ahora cuentame tu que haces por aqui no digas que ese otro no te hizo feliz porque en mi t'
Target data:  'o podras ahora cuentame tu que haces por aqui no digas que ese otro no te hizo feliz porque en mi ti'


In [14]:
#imprimimos el tensor del dataset
print(dataset)
#Hyper-Parametros para entrenamiento  de una rede neuronal 
#   -los datos se agrupan en batch
BATCH_SIZE= 32
#    -Tamaño de memoria disponible 
BUFFER_SIZE=10000
dataset= dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
print (dataset)
#En el tensor dataset disponemos los datos de entrenamiento
#con agrupamienttos (batches) compuestos de 64 parejas de secuencias 
#de 100 integers de 64 bits que representan el carácter correspondiente 
#en el vocabulario.

<MapDataset shapes: ((100,), (100,)), types: (tf.int64, tf.int64)>
<BatchDataset shapes: ((32, 100), (32, 100)), types: (tf.int64, tf.int64)>


##P4.Construcción del modelo RNN

---
Para construir el modelo usaremos tf.keras.Sequential. Usaremos una versión mínima de RNN, que contenga solo una capa LSTM y 3 capas.


In [24]:
#como es un problema de clasificación estándar 
#para el que debemos definir la función de Lossy el optimizador.
def lossy(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

def create_model(vocab_size, embedding_dim, rnn_units, batch_size):
  #creando el modelo
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(rnn_units,
                         return_sequences=True,
                         stateful=True,
                         recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)                               
  ])
  #En cuanto al optimizador usaremos tf.keras.optimizers.Adam 
  #con los argumentos por defecto del optimizador Adam. 
  model.compile(optimizer='adam',
              loss=tf.keras.losses.sparse_categorical_crossentropy(),
              metrics=['accuracy'])
  return model
vocab_size= len(vocab)
#dimensiones de los vectores que tendrá la capa.
embedding_dim= 256
#cantidad de neuronas
rnn_units=1024
#creamos nuestra red neuronal RNN
model=create_model(vocab_size   =vocab_size,
                  embedding_dim =embedding_dim,
                  rnn_units     =rnn_units,
                  batch_size    =BATCH_SIZE)
#summary()para visualizar la estructura del modelo
model.summary()
#resultados=  -La capa LSTM consta más de 5 millones de parametros)

TypeError: ignored

###P4.1 Creando chekpoints

---
una técnica de tolerancia de fallos para procesos cuyo tiempo de ejecución es muy largo. La idea es guardar una instantánea del estado del sistema periódicamente para recuperar desde ese punto la ejecución en caso de fallo del sistema.

---
los crearemos en google drive para mejorar la capacidad de reentrenamiento de la red


In [16]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [17]:
checkpoint_dir='/content/gdrive/MyDrive/Colab Notebooks/Proyecto final/checkpoints'
checkpoint_prefix= os.path.join(checkpoint_dir,"cp_{epoch:04d}.ckpt")


cp_callback=tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_prefix,
                                               monitor='loss',
                                               verbose=1,
                                               save_weights_only=True,
                                               save_best_only=True,
                                               mode='auto')


###P4.2 entrenando

####P4.2a entrenando para usar chekpoints


---

*   https://keras.io/api/callbacks/model_checkpoint/
*   https://towardsdatascience.com/checkpointing-deep-learning-models-in-keras-a652570b8de6

In [20]:
EPOCHS=500
history=model.fit(dataset, 
                  epochs=EPOCHS, 
                  verbose=1,
                  callbacks=[cp_callback])

Epoch 1/500


ValueError: ignored

##P5. Generando texto nuevo usando la RNN

In [ ]:
#creamos un modelo tomando como base el ultimo checkpoint
model = create_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1,None]))

In [ ]:
#funcion para generar texto
def generate_text(model, start_string):
  #definimos cuantos tensores/cantidad de texto generaremos
  num_generate=500
  #convertimos el texto en números
  input_eval=[char2idx[s] for s in start_string]
  input_eval= tf.expand_dims (input_eval,0)
  text_generated = []

  temperature = 0.5  #(0.0 a  1) entre más alta la temperatura más creatividad al modelo, pero tambien más errores ortograficos.
  model.reset_states() #bucle para generar caracteres, mediante predicciones
  for i in range(num_generate):
    predictions = model(input_eval)
    predictions = tf.squeeze(predictions, 0)
    predictions = predictions / temperature
    predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
    input_eval= tf.expand_dims([predicted_id],0)
    text_generated.append (idx2char[predicted_id])
  
  return (start_string+ ''.join(text_generated))


###P5.1 generando texto 

In [ ]:
print(generate_text(model, start_string=u"amor"))

amores nuevos se da golpe de pecho pero es pa apantalla porque cuando esta en temple se pone es a llamar pregunta en las esquinas me busca como loca no puedes volar conmigo aunque mis sueños se iran contigo todo de cabeza esta es mi cancion de despedida sera lo mejor para los dos pues no soy el dueño de tu vida y lo mas grave no se porque no lo entiende el corazon dime como hago para explicarle a mi alma envuelta entre la fe que aunque sea dificil olvidarte debo intentarlo por que creo que nunca te 


##P6.exportando modelo

---
Guardamos y Serializamos el Modelo (con esto ya podemos vender nuestro modelo de predicción de texto según lo aprendido por nuestra RNN).


In [ ]:
from keras.models import model_from_json
import os
dir_export= '/content/gdrive/MyDrive/Colab Notebooks/Proyecto final/Modelo/'
#dir_export= os.path.join(dir_drive)
# Serializamos el modelo en forma JSON
model_json = model.to_json()
with open(os.path.join(dir_export,'RNN_Solo_Letras_PLN_Vallenato.json'), 'w') as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights(os.path.join(dir_export,'RNN_Solo_Letras_PLN_Vallenato.hdf5'))
model.save(os.path.join(dir_export,'RNN_Solo_Letras_PLN_Vallenato.h5'))
print("modelo salvado en Drive de google")

modelo salvado en Drive de google


##P7.Cargando un modelo serializado

In [ ]:
!wget https://github.com/Moisesdv10/Deep-Learning/blob/main/Modelos_PLN_Vallenato/RNN_Solo_Letras_PLN_Vallenato.h5\
      -O modelRNN_vallenato.h5

--2021-11-27 17:33:38--  https://github.com/Moisesdv10/Deep-Learning/blob/main/Modelos_PLN_Vallenato/RNN_Solo_Letras_PLN_Vallenato.h5
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘modelRNN_vallenato.h5’

modelRNN_vallenato.     [ <=>                ] 164.01K  --.-KB/s    in 0.07s   

2021-11-27 17:33:38 (2.30 MB/s) - ‘modelRNN_vallenato.h5’ saved [167948]



####P7.1a descargamos el modelo usando PYRIND & URLLIB (OPCIONAL)

In [ ]:
!pip install pyprind

In [ ]:
def reporthook(count, block_size, total_size):
    global start_time
    if count == 0:
        start_time = time.time()
        return
    duration = time.time() - start_time
    progress_size = int(count * block_size)
    speed = progress_size / (1024.**2 * duration)
    percent = count * block_size * 100. / total_size
    sys.stdout.write("\r%d%% | %d MB | %.2f MB/s | %d segundos transcurrido" %
                    (percent, progress_size / (1024.**2), speed, duration))
    sys.stdout.flush()

import urllib.request
url_github_Model='https://github.com/Moisesdv10/Deep-Learning/blob/main/Modelos_PLN_Vallenato/RNN_Solo_Letras_PLN_Vallenato.h5?raw=true'
urllib.request.urlretrieve(url_github_Model,
                           'RNN_vallenato_model_3.h5', 
                           reporthook)
#model.save(os.path.join("./dnn/","RNN_vallenato_model_2.h5"))

100% | 60 MB | 3.29 MB/s | 18 segundos transcurrido

('RNN_vallenato_model_3.h5', <http.client.HTTPMessage at 0x7feb4d958510>)

In [ ]:
# Guardar el Modelo
model.save('path_to_my_model.h5')

# Recrea exactamente el mismo modelo solo desde el archivo
new_model = keras.models.load_model('path_to_my_model.h5')

In [ ]:
import numpy as np

# Verifique que el estado esté preservado
new_predictions = new_model.predict(x_test)
np.testing.assert_allclose(predictions, new_predictions, rtol=1e-6, atol=1e-6)

# Tenga en cuenta que el estado del optimizador también se conserva:
# puede reanudar el entrenamiento donde lo dejó.

In [ ]:
# Exportar el modelo a 'SavedModel'
keras.experimental.export_saved_model(model, 'path_to_saved_model')

# Recrea exactamente el mismo modelo
new_model = keras.experimental.load_from_saved_model('path_to_saved_model')

# Verifique que el estado esté guardado
new_predictions = new_model.predict(x_test)
np.testing.assert_allclose(predictions, new_predictions, rtol=1e-6, atol=1e-6)

# Tenga en cuenta que el estado del optimizador también se conserva:
# puede reanudar el entrenamiento donde lo dejó.

###P7.2 instanciamos el modelo descargado

In [ ]:
new_model = tf.keras.models.load_model('/content/RNN_vallenato_model_3.h5')

ValueError: ignored

In [ ]:
df2 = pd.read_csv("https://raw.githubusercontent.com/Moisesdv10/Deep-Learning/main/Proyecto_Final/data_vocab.csv")
df2.head()

,num,vocab
0,0,
1,1,0
2,2,1
3,3,2
4,4,4


In [ ]:
#funcion para generar texto
def generate_text(model, start_string):
  #definimos cuantos tensores/cantidad de texto generaremos
  num_generate=500
  #convertimos el texto en números
  input_eval  = [char2idx[s] for s in start_string]
  input_eval  = tf.expand_dims (input_eval,0)
  text_generated = []

  temperature = 0.5  #(0.0 a  1) entre más alta la temperatura más creatividad al modelo, pero tambien más errores ortograficos.
  model.reset_states() #bucle para generar caracteres, mediante predicciones
  for i in range(num_generate):
    predictions = model(input_eval)
    predictions = tf.squeeze(predictions, 0)
    predictions = predictions / temperature
    predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
    input_eval= tf.expand_dims([predicted_id],0)
    text_generated.append (idx2char[predicted_id])
  
  return (start_string+ ''.join(text_generated))


In [ ]:
print(generate_text(new_model, start_string=u"amor"))

NameError: ignored